In [2]:
from utilities import *
from core.initial import Insertions
from core.solver import CGCPSolver
from docplex.cp.model import CpoModel
from docplex.mp.model import Model
import pandas as pd

In [3]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()
initial = Insertions(model)
initial.solve()
s = CGCPSolver(initial.sol)

In [4]:
rmp = Model(name='Restricted_Master_Problem')

rmp.trips = s.trips
rmp.duties = s.duties

rmp.trip_duty_arr = rmp.binary_var_matrix(s.trips, s.duties)

In [5]:
rmp.duty_arr = rmp.binary_var_list(rmp.duties, name='duty')

In [6]:
for trip in rmp.trips:
    duty_ct = rmp.sum(rmp.trip_duty_arr[(trip, duty)] * rmp.duty_arr[duty.ID] for duty in rmp.duties) == 1
    rmp.add_constraint(duty_ct)

In [9]:
rmp.minimize(rmp.sum(rmp.duty_arr[duty.ID] for duty in rmp.duties))

In [10]:
rmp.solve()

Error: Model has non-convex quadratic constraint, index=0
